In [1]:
# Project 1 v3
# Bringing it all together

In [18]:
# imports
# Required libraries to be installed: Pillow, OpenCV
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display
import PIL.Image
import io

ModuleNotFoundError: No module named 'sklearn'

In [3]:
# helper functions
def show_image(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def read_image(index, source, resize):
    image = cv.imread(source[index])
    return cv.resize(image, (0, 0), fx=resize, fy=resize)

In [4]:
# I tried with multiple images in the beginning
template_images = glob.glob(os.path.join('./', "template_*.jpg")) 
char_to_index = {'A': 0, 'B': 1, 'C': 2, 'D': 3}


In [5]:
# helper variables
template_image_full = read_image(0, template_images, 1)
template_image = cv.resize(template_image_full, (0, 0), fx=0.6, fy=0.6)
h, w, _ = template_image.shape
template_image = template_image[int(0.36 * h):h, :]
# show_image(template_image)

In [6]:
def get_first_grid(image):
    image_h, image_w, channels = image.shape
    return image[int(0.32 * image_h):int(image_h * 0.78), int(image_w * 0.222): int(image_w * 0.367)]

def get_second_grid(image):
    image_h, image_w, channels = image.shape
    return image[int(0.32 * image_h):int(image_h * 0.79), int(image_w * 0.75): int(image_w * 0.90)]

In [7]:
# I initially played with it, taking the first best features etc
# still, the best result is with the lab code, just with nfeatures = 10k instead of 5k
def warp_image(template, query):
    img1 = template
    img2 = query
    # create ORB object
    orb = cv.ORB_create(nfeatures=10000)
    # get the keypoints and the corresponding descriptors
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None) 
    # create BFMatcher object
    # matcher takes normType, which is set to cv2.NORM_L2 for SIFT and SURF, cv2.NORM_HAMMING for ORB, FAST and BRIEF
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des2, des1) # query_image, train_image
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    # points template from img1, the template image
    points_template = np.zeros((len(matches), 2), dtype=np.float32)
    # points_query from img2, the query image
    points_query = np.zeros((len(matches), 2), dtype=np.float32)

    for i,m in enumerate(matches):
        points_template[i,:] = kp1[m.trainIdx].pt
        points_query[i,:] = kp2[m.queryIdx].pt

    H,mask = cv.findHomography(points_query, points_template, cv.RANSAC)

    # use homography to get the aligned image 
    height, width, _ = template.shape # the shape with respect to the template image
    aligned_image2 = cv.warpPerspective(query, H, (width, height), flags=cv.INTER_NEAREST)

    return aligned_image2

In [8]:
# Given any of the 2 tables, it finds it
# The positions are hard coded. I initially detected the lines and columns, but before I could perfect
# it this yielded good results so I left it like this. It's a bit dirty but it works
def find_x_from_table(table):
    table_gray = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
    image = np.dstack((table_gray, table_gray, table_gray))
    x_color = (0, 255, 0)  # plot a patch containing an X with green color
    blank_color = (0, 0, 255)  # plot a patch containing a blank with red color 
    x_positions = [0] * 15
    w, h, _ = image.shape
    cell_height = int(h / 5)
    cell_width = int(w / 4)
    
    y_mins = [5, 24, 40, 57, 75, 90, 108, 125, 142, 159, 176, 192, 210, 227, 245]
    
    for i in range(0, 15):
        colors = [blank_color] * 4
            
        y_min = y_mins[i]
        y_max = y_min + 10
            
        x1_min = 4
        x1_max = 20
        
        x2_min = 27
        x2_max = 42
        
        x3_min = 48
        x3_max = 64
        
        x4_min = 71
        x4_max = 84
        
        patch1 = image[y_min:y_max,x1_min:x1_max].copy().mean()
        patch2 = image[y_min:y_max,x2_min:x2_max].copy().mean()
        patch3 = image[y_min:y_max,x3_min:x3_max].copy().mean()
        patch4 = image[y_min:y_max,x4_min:x4_max].copy().mean()
        patches = [patch1, patch2, patch3, patch4]
        
        # Get the indices of maximum element in numpy array
        min_value = min(patches)
        index = patches.index(min_value)
        colors[index] = x_color
        x_positions[i] = index
        cv.rectangle(image, (x1_min, y_min), (x1_max, y_max), color=colors[0], thickness=1)
        cv.rectangle(image, (x2_min, y_min), (x2_max, y_max), color=colors[1], thickness=1)
        cv.rectangle(image, (x3_min, y_min), (x3_max, y_max), color=colors[2], thickness=1)
        cv.rectangle(image, (x4_min, y_min), (x4_max, y_max), color=colors[3], thickness=1)
# I return both the image and the position so that I can show the image if the positions do not match
    return (x_positions, image)

In [9]:
# reads and returns an dictionary of arrays with the ground truth answers
def get_ground_truth_answers(path):
    file_names = glob.glob(os.path.join(path, '*.txt')) 
    phisycs = []
    informatics = np.empty((4,), dtype=object)
    physics = np.empty((4,), dtype=object)
    ground_truth = {}
    for file in file_names:
        data = np.loadtxt(file, dtype=str)
        option = data[0][0]
        variant = int(data[0][1])
        if (option == 'I'):
            informatics[variant - 1] = data[1:-1]
        if (option == 'F'):
            physics[variant - 1] = data[1:-1]
    ground_truth['F'] = physics
    ground_truth['I'] = informatics
    return ground_truth

In [10]:
# Concatenates both table readings and returns them as an array
def get_answers_from_image(image):
    table1 = get_first_grid(image)
    table2 = get_second_grid(image)
    
    table_x1 = find_x_from_table(table1)
    table_x2 = find_x_from_table(table2)
#     show_image(table_x1[1])
#     show_image(table_x2[1])
    results = table_x1[0] + table_x2[0]   
    return (results, table_x1[1], table_x2[1])


In [11]:
# answers = computed answers from the image
# options = I / F
# variant = 1, 2, 3 or 4
# ground_truth = returned from get_ground_truth_answers(path), a dictionary of arrays
def calculate_grade(answers, ground_truth, option, variant):
    grade = 0
    for i in range (0, 30):
        # variant - 1 so that we map 1..4 to 0..3
        ground_truth_answer = ground_truth[option][variant - 1][i][1] 
        if (answers[i] == char_to_index[ground_truth_answer]):
            grade += 0.3
    return round(grade + 1, 2)

In [13]:
# Scenario 1
# In each scenario I read the variables from scratch, to be sure

# ground truth, the 8 possible grading scenarios
ground_truth_answers_path = '../Files/ground-truth-correct-answers'
ground_truth = get_ground_truth_answers(ground_truth_answers_path)

# images folder
images = glob.glob('./1.scanned/*.jpg') 
# sort them by number
try: 
    images.sort(key = lambda x: int(x.split('/')[-1].split('_')[0]))
    total = len(images)
except:
    print("An exception occurred")
    
print("Starting scenario 1 on", total, "files")
# answers file
answers_file = open('dumitriu_andrei_task1.txt', 'w+')

for i in range(0, total):
    try:
        image_name = images[i].split('/')[-1].split('.')[0]
        option = image_name[-2]
        variant = int(image_name[-1])
        image_number = int(images[i].split('/')[-1].split('.')[0].split('_')[0])

        image = read_image(i, images, 1)
        # warping the image two times, to help improve accuracy
        warped1 = warp_image(template_image, image)
        warped = warp_image(template_image, warped1)

        # I had a different structure in the code before and resized twice
        # I tried replacing it with one resize of 0.25 but it yields worse results
        resized = cv.resize(warped, (0,0), fx=0.5, fy=0.5)
        image = cv.resize(resized, (0,0), fx=0.5, fy=0.5)

        answers = get_answers_from_image(image)[0]
        grade = calculate_grade(answers, ground_truth, option, variant)

        written_string = str(image_name + ".jpg" + "    " + str(grade))
        answers_file.write(written_string)
        answers_file.write('\n')
#         print(image_name)
    except:
        print("An exception occurred") # careful, this also blocks kernel intrerrupt

answers_file.close()
print("Done.")

Starting scenario 1 on 150 files
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [12]:
# Scenario 2
# In each scenario I read the variables from scratch, to be sure

# ground truth, the 8 possible grading scenarios
ground_truth_answers_path = '../Files/ground-truth-correct-answers'
ground_truth = get_ground_truth_answers(ground_truth_answers_path)

# images folder
images = glob.glob("./2.rotated+perspective/*.jpg") 

# sort them by number
try: 
    images.sort(key = lambda x: int(x.split('/')[-1].split('_')[0]))
    total = len(images)
except:
    print("An exception occurred")
# answers file
answers_file = open('dumitriu_andrei_task2.txt', 'w+')

print("Starting scenario 2 on", total, "files")
for i in range(0, total):
    try:
        image_name = images[i].split('/')[-1].split('.')[0]
        option = image_name[-2]
        variant = int(image_name[-1])
        image_number = int(images[i].split('/')[-1].split('.')[0].split('_')[0])

        image = read_image(i, images, 1)
        # warping the image two times, to help improve accuracy
        warped1 = warp_image(template_image, image)
        warped = warp_image(template_image, warped1)

        # I had a different structure in the code before and resized twice
        # I tried replacing it with one resize of 0.25 but it yields worse results
        resized = cv.resize(warped, (0,0), fx=0.5, fy=0.5)
        image = cv.resize(resized, (0,0), fx=0.5, fy=0.5)

        answers = get_answers_from_image(image)[0]

        table1 = get_answers_from_image(image)[1]
        table2 = get_answers_from_image(image)[2]

        grade = calculate_grade(answers, ground_truth, option, variant)

        written_string = str(image_name + ".jpg" + "    " + str(grade))
        answers_file.write(written_string)
        answers_file.write('\n')
#         print(image_name)
    except:
        print("An exception occurred") # careful, this also blocks kernel intrerrupt
    
answers_file.close()
print("Done.")

Starting scenario 2 on 300 files
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
2

In [17]:
# Scenario 3
# In each scenario I read the variables from scratch, to be sure

# ground truth, the 8 possible grading scenarios
ground_truth_answers_path = '../Files/ground-truth-correct-answers'
ground_truth = get_ground_truth_answers(ground_truth_answers_path)

# images folder
images = glob.glob("./3.no_annotation/*.jpg") 

# sort them by number
try:
    images.sort(key = lambda x: int(x.split('/')[-1].split('_')[0].split('.')[0]))
    total = len(images)
except:
    print("An exception occurred")
    
# answers file
answers_file = open('dumitriu_andrei_task3.txt', 'w+')

count = 0 # the number of correctly read answers
print("Starting scenario 3 on", total, "files")
for i in range(0, total):
    try:
        # for real images
        image_name = images[i].split('/')[-1].split('.')[0]   
        image = read_image(i, images, 1)
        # warping the image two times, to help improve accuracy
        warped1 = warp_image(template_image, image)
        warped = warp_image(template_image, warped1)
        # I had a different structure in the code before and resized twice
        # I tried replacing it with one resize of 0.25 but it yields worse results
        resized = cv.resize(warped, (0,0), fx=0.5, fy=0.5)
        image = cv.resize(resized, (0,0), fx=0.5, fy=0.5)    

        answers = get_answers_from_image(image)[0]    
        table1 = get_answers_from_image(image)[1]
        table2 = get_answers_from_image(image)[2]

        guessed_grade = 0
        guessed_option = 'F'
        guessed_variant = 1
        max_grade = 0.0
        for j in range (1, 5):
            f_grade = calculate_grade(answers, ground_truth, 'F', j)
            i_grade = calculate_grade(answers, ground_truth, 'I', j)
            if (max_grade < f_grade):
                guessed_option = 'F'
                guessed_variant = j
                max_grade = f_grade

            if (max_grade < i_grade):
                guessed_option = 'I'
                guessed_variant = j
                max_grade = i_grade

        guessed_grade = float(calculate_grade(answers, ground_truth, guessed_option, guessed_variant))

        written_string = str(image_name + ".jpg" + "    " + str(guessed_grade))
        answers_file.write(written_string)
        answers_file.write('\n')
        print(image_name)
    except:
        print("An exception occurred") # careful, this also blocks kernel intrerrupt
answers_file.close()
print("Done.")

Starting scenario 3 on 450 files
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
26

In [49]:
# Scenario 4
# In each scenario I read the variables from scratch, to be sure

# ground truth, the 8 possible grading scenarios
ground_truth_answers_path = '../Files/ground-truth-correct-answers'
ground_truth = get_ground_truth_answers(ground_truth_answers_path)

# images folder
images = glob.glob("./4.handwritten/*.jpg") 

# sort them by number
try:
    images.sort(key = lambda x: int(x.split('/')[-1].split('_')[0].split('.')[0]))
    total = len(images)
except:
    print("An exception occurred")
    
# answers file
answers_file = open('dumitriu_andrei_task4.txt', 'w+')

count = 0 # the number of correctly read answers
print("Starting scenario 3 on", total, "files")
for i in range(0, total):
    j = 0


139


92.66666666666667

In [ ]:
# Scenario 3 TEST
# In each scenario I read the variables from scratch, to be sure

# ground truth, the 8 possible grading scenarios
ground_truth_answers_path = './Files/ground-truth-correct-answers'
ground_truth = get_ground_truth_answers(ground_truth_answers_path)

# images folder
images = glob.glob("./Simulation/scenario_3/*.jpg") 

# sort them by number
try:
    images.sort(key = lambda x: int(x.split('/')[-1].split('_')[0].split('.')[0]))
    total = len(images)
except:
    print("An exception occurred")
    
# answers file
answers_file = open('dumitriu_andrei_task3.txt', 'w+')

count = 0 # the number of correctly read answers
for i in range(0, total):
    # for real images
    image_name = images[i].split('/')[-1].split('.')[0]
#     option = image_name[-2]
#     variant = int(image_name[-1])  
    image_number = int(images[i].split('/')[-1].split('.')[0].split('_')[0])
    if (image_number > 150 and image_number < 301):
        image_number -= 150
    if (image_number > 300):
        image_number -= 300

    # for test
    file = os.path.join('./Files/images', 'image_%s.txt' % image_number)
    data = np.loadtxt(file, dtype=str)
    option = data[0][0]
    variant = int(data[0][1])
    real_grade = float(round(int(data[-1][1]) * 0.3 + 1, 2))
    
    
#     print(real_grade)
#     print(image_name)
#     print(option)
#     print(variant)
#     print(image_number)
    
    # end of test
    

    
    
    image = read_image(i, images, 1)
    # warping the image two times, to help improve accuracy
    warped1 = warp_image(template_image, image)
    warped = warp_image(template_image, warped1)
    
    # I had a different structure in the code before and resized twice
    # I tried replacing it with one resize of 0.25 but it yields worse results
    resized = cv.resize(warped, (0,0), fx=0.5, fy=0.5)
    image = cv.resize(resized, (0,0), fx=0.5, fy=0.5)    
    
    answers = get_answers_from_image(image)[0]    

    
    table1 = get_answers_from_image(image)[1]
    table2 = get_answers_from_image(image)[2]
    
    guessed_grade = 0
    guessed_option = 'F'
    guessed_variant = 1
    max_grade = 0.0
    for i in range (1, 5):
        f_grade = calculate_grade(answers, ground_truth, 'F', i)
        i_grade = calculate_grade(answers, ground_truth, 'I', i)
#         print(type(i_grade))
#         print("fizica:", i, "nota", f_grade)
#         print("info:", i, "nota", i_grade)
        if (max_grade < f_grade):
            guessed_option = 'F'
            guessed_variant = i
            max_grade = f_grade
            
        if (max_grade < i_grade):
            guessed_option = 'I'
            guessed_variant = i
            max_grade = i_grade
            
    guessed_grade = float(calculate_grade(answers, ground_truth, guessed_option, guessed_variant))
    
    written_string = str(image_name + "    " + str(grade))
    answers_file.write(written_string)
    answers_file.write('\n')
    
    # verify, can de deleted afterwards
    verified = verify_grade_file(image_number, grade)
    if (guessed_grade == real_grade):
        count += 1
        print(image_name)
    else:
        print('\n')
        print("image:", image_name)
        print("image number:", image_number)
        print("real option:", option)
        print("real variant:", variant)
        print("real grade:", real_grade)
        print("guessed option:", guessed_option)
        print("guessed variant:", guessed_variant)
        print("guessed grade:", guessed_grade)
        print('\n')
    # end of deletion
    if (image_number == 150):
        print ("% for first 150 images")
        print (count * 100 / 150)
    if (image_number == 300):
        print ("% for first 300 images")
        print (count * 100 / 300)

answers_file.close()
print ("% for all 450 images")
print (count * 100 / total)